**Imports**

In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_score, recall_score, roc_auc_score
from imblearn.pipeline import Pipeline as ImbPipeline
import pandas as pd
import sys

sys.path.append('C:\\Coding\\customer-churn-prediction\\src')
from utils import download_telco_churn_dataset, split_test_train
from preprocessing import get_preproc

RND_SEED = 42

**Getting Preprocessing and Data**

In [2]:
telco = pd.read_csv("../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv")
X_train, X_test, y_train, y_test = split_test_train(telco)

preprocessing = get_preproc()

# Models

**Help Function**

Функция для оценки различных моделей на тестовой и тернировочных данных.

In [3]:
from sklearn.utils import class_weight

def fit_and_evaluate(models, cv=10):
    # Крафтим пайплайны для каждой модели
    pipelines = {}

    for name, model in models:
        pipelines[name] = (ImbPipeline([
            ("preproc", preprocessing.named_steps['preproc']),
            ("smote", preprocessing.named_steps['smote']),
            ("model", model)
        ]))

    metrics = {}

    # Оценивать модели будем по roc_auc score
    for name, model in pipelines.items():
        # Оценим сырые модельки на CV
        roc_auc_cv = cross_val_score(
            model, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1).mean()

        model.fit(X_train, y_train)

        y_pred_test = model.predict(X_test)

        # Metrics evaluating
        roc_auc_test = roc_auc_score(y_test, y_pred_test)

        metrics[name] = {
            'roc_auc_test': roc_auc_test,
            'roc_auc_cv': roc_auc_cv,
        }

    sorted_metrics = dict(
        sorted(metrics.items(), key=lambda item: -item[1]['roc_auc_cv']))

    print(f"ТОП МОДЕЛЕЙ по ROC-AUC:")
    for name, metric in sorted_metrics.items():
        print(
            "\n" + f"{name} : \nAUC: {metric['roc_auc_cv']}")

## Models Comparation

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state=RND_SEED, verbose=-1)

lr = LogisticRegression(random_state=RND_SEED, penalty='l2')

lin_svc = LinearSVC(C=1, random_state=RND_SEED)

gb = GradientBoostingClassifier(random_state=RND_SEED)

svc = SVC(C=1, random_state=RND_SEED)

rnd_forest = RandomForestClassifier(random_state=RND_SEED, n_jobs=-1)

xgb = XGBClassifier(random_state=RND_SEED, n_jobs=-1)

models = [
    ("Logistic Regression L2", lr),
    ("LightGBM", lgbm),
    ("RND Forest", rnd_forest),
    ("XGB Classifier", xgb),
    ("Gradient Boosting", gb),
    ("Linear SVC", lin_svc),
    ("SVC", svc),
]

In [ ]:
fit_and_evaluate(models)

c:\Coding\customer-churn-prediction\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Непосредственно в этой задаче мне важен `Recall` на положительном класса, то есть процент верно определенных `Churn`.

## Lin SVC + GB + Logist Regr

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, uniform, randint


def fine_tuning_models(models_data):
    for name, model_data in models_data.items():
        models_data[name] = {
            "pipeline": ImbPipeline([
                ("preproc", preprocessing.named_steps['preproc']),
                ("smote", preprocessing.named_steps['smote']),
                ("model", model_data["model"]),
            ]),
            "param_distrib": model_data["param_disturb"],
        }

        roc_auc_losses = cross_val_score(
            models_data[name]["pipeline"], X_train, y_train, cv=10, n_jobs=-1, scoring='roc_auc')
        print(f'{name} ROC-AUC:\n{pd.Series(roc_auc_losses).mean()}')

    print('=' * 50 + "Tuned models!!!" + '=' * 50)

    best_models = {}

    for name, model_data in models_data.items():
        rnd_search = RandomizedSearchCV(model_data["pipeline"], param_distributions=model_data["param_distrib"],
                                        n_iter=50, cv=5, n_jobs=-1, random_state=RND_SEED, scoring='roc_auc')
        rnd_search.fit(X_train, y_train)

        roc_auc_losses_cv = rnd_search.best_score_

        print(f'{name} AUC:\n{pd.Series(roc_auc_losses_cv).mean()}')

        best_models[name] = rnd_search.best_estimator_

    return best_models
        
    

In [ ]:
pd_lin_svc = {
    "model__tol": uniform(1e-6, 1e-4),
    "model__C": loguniform(0.01, 5),
    "model__fit_intercept": [False],
}

pd_gb = {
    "model__learning_rate": loguniform(0.05, 0.1),
    "model__n_estimators": randint(100, 200),
    "model__max_depth": randint(3, 5),
    "model__max_features": ['sqrt', 'log2', None],
}

pd_lr = [
    {
        "model__penalty": ['l2'],
        "model__tol": uniform(1e-6, 1e-4),
        "model__C": loguniform(0.01, 2),
        "model__max_iter": randint(50, 500),
        "model__solver": ['lbfgs', 'liblinear'],
    }
]

models_data = {
    "Logistic Regression": {
        "model": lr,
        "param_disturb": pd_lr,
    },
    "Gradient Boosting": {
        "model": gb,
        "param_disturb": pd_gb,
    },
    "Linear SVC": {
        "model": lin_svc,
        "param_disturb": pd_lin_svc,
    },
}

In [ ]:
best_models = fine_tuning_models(models_data)

Тут лучше всего себя показал градиентный бустинг!

In [ ]:
gb_tuned = best_models["Gradient Boosting"]
lr_tuned = best_models["Logistic Regression"]
lin_svc_tuned = best_models["Linear SVC"]

from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import numpy as np

y_scores = gb_tuned.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, y_scores)
average_precision = average_precision_score(y_test, y_scores)

# Строим график
plt.figure(figsize=(10, 6))
plt.plot(recall, precision, linewidth=2, label=f'Gradient Boosting (AP = {average_precision:.2f})')

y_scores = lr_tuned.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, y_scores)
average_precision = average_precision_score(y_test, y_scores)

plt.plot(recall, precision, linewidth=2, label=f'Linear Regression (AP = {average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)
plt.axis([0, 1, 0, 1])
plt.show()

In [ ]:
from sklearn.model_selection import learning_curve, cross_val_predict
from sklearn.metrics import confusion_matrix

y_pred_cv = cross_val_predict(lr_tuned, X_train, y_train, cv=10, n_jobs=-1)
cm_cv = confusion_matrix(y_train, y_pred_cv)

cm_cv

In [ ]:
from sklearn.metrics import precision_recall_curve

y_pred_proba = gb_tuned.predict_proba(X_test)[:, 1]

precision, recall, threshold = precision_recall_curve(y_test, y_pred_proba)

optimal_idx = np.argmax(recall > 0.85)
optimal_threshold = threshold[optimal_idx]

y_pred_otimised = (y_pred_proba >= optimal_threshold).astype(int)